In [12]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

XDIM = 15
TRAIN_SLOPE = 1
EVAL_SLOPE = 5
GAMING = 1
EPSILON = 0.1
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10

# Utils

In [13]:
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    data = torch.cat((X, Y), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, :2]
    Y = data[:, 2]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

# Dataset

In [14]:
def load_spam_dataset():
    torch.manual_seed(0)
    np.random.seed(0)
    path = r"C:\Users\sagil\Desktop\nir project\tip_spam_data\IS_journal_tip_spam.arff"
    data, meta = arff.loadarff(path)
    df = pd.DataFrame(data)
    most_disc = ['qTips_plc', 'rating_plc', 'qEmail_tip', 'qContacts_tip', 'qURL_tip', 'qPhone_tip', 'qNumeriChar_tip', 'sentistrength_tip', 'combined_tip', 'qWords_tip', 'followers_followees_gph', 'qunigram_avg_tip', 'qTips_usr', 'indeg_gph', 'qCapitalChar_tip', 'class1']
    df = df[most_disc]
    df["class1"].replace({b'spam': -1, b'notspam': 1}, inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)

    Y = df['class1'].values
    X = df.drop('class1', axis = 1).values
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    return torch.from_numpy(X), torch.from_numpy(Y)

def gen_custom_data(N, pos_ranges, neg_ranges):
    """
    pos_ranges: a tuple of tensors of length XDIM.
    (scales tensor, offsets tensor)
    """
    torch.manual_seed(0)
    np.random.seed(0)

    pos_samples_num = N//2
    neg_samples_num = N - pos_samples_num
    posX = torch.rand((pos_samples_num, XDIM))*pos_ranges[0] + pos_ranges[1]
    negX = torch.rand((neg_samples_num, XDIM))*neg_ranges[0] + neg_ranges[1]
    
    X = torch.cat((posX, negX), 0)
    Y = torch.unsqueeze(torch.cat((torch.ones(len(posX)), -torch.ones(len(negX))), 0), 1)

    X, Y = shuffle(X, Y)
    return X, Y

# CCP classes

In [15]:
class CCP:
    def __init__(self, x_dim, funcs):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.xt = cp.Parameter(x_dim)
        self.r = cp.Parameter(x_dim)
        self.w = cp.Parameter(x_dim)
        self.b = cp.Parameter(1)
        self.slope = cp.Parameter(1)
        self.v = cp.Parameter(x_dim)

        target = self.x@self.f_derivative(self.xt, self.w, self.b, self.slope)-self.g(self.x, self.w, self.b, self.slope)-self.c(self.x, self.r, self.v)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(target), constraints)
        
    def ccp(self, r):
        """
        numpy to numpy
        """
        self.xt.value = r
        self.r.value = r
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        cnt = 0
        while diff > 0.0001 and cnt < 10:
            cnt += 1
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)
        return self.x.value
    
    def optimize_X(self, X, w, b, slope, v):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        slope = np.full(1, slope)
        X = X.numpy()
        
        self.w.value = w
        self.b.value = b
        self.slope.value = slope
        self.v.value = v
        
        return torch.stack([torch.from_numpy(self.ccp(x)) for x in X])

In [16]:
class DELTA():
    
    def __init__(self, x_dim, funcs, v):
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))
        self.f_der = cp.Parameter(x_dim, value = np.random.randn(x_dim))

        target = self.x@self.f_der-self.g(self.x, self.w, self.b, TRAIN_SLOPE)-self.c(self.x, self.r, v)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b, self.f_der],
                                variables=[self.x])
        
    def optimize_X(self, X, w, b, F_DER):
        return self.layer(X, w, b, F_DER)[0]

# Gain & Cost functions

In [17]:
def score(x, w, b):
    return x@w + b

def f(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) + 1)]), 2)

def g(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) - 1)]), 2)

def c_true(x, r, v):
    print(GAMING, EPSILON)
    return 2*(1./GAMING)*(EPSILON*cp.sum_squares(x-r) + (1-EPSILON)*cp.pos((x-r)@v))

def c(x, r, v):
    print(GAMING)
    return 2*(1./GAMING)*(0.01*cp.sum_squares(x-r) + (0.99)*cp.pos((x-r)@v))

def f_derivative(x, w, b, slope):
    return 0.5*cp.multiply(slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1)), w)

funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}
funcs_val = {"f": f, "g": g, "f_derivative": f_derivative, "c": c_true, "score": score}

# Data generation

In [18]:
X, Y = load_spam_dataset()

assert(len(X[0]) == XDIM)
X, Y, Xval, Yval = split_data(X, Y, 0.4)
Xval, Yval, Xtest, Ytest = split_data(Xval, Yval, 0.5)

path = "C:/Users/sagil/Desktop/nir project/tip_spam_data"
pd.DataFrame(X.numpy()).to_csv(path + '/X.csv')
pd.DataFrame(Y.numpy()).to_csv(path + '/Y.csv')
pd.DataFrame(Xval.numpy()).to_csv(path + '/Xval.csv')
pd.DataFrame(Yval.numpy()).to_csv(path + '/Yval.csv')
pd.DataFrame(Xtest.numpy()).to_csv(path + '/Xtest.csv')
pd.DataFrame(Ytest.numpy()).to_csv(path + '/Ytest.csv')

print("percent of positive samples: {}%".format(100 * len(Y[Y == 1]) / len(Y)))

percent of positive samples: 49.92934526613283%


# Model

In [19]:
class MyStrategicModel(torch.nn.Module):
    def __init__(self, x_dim, funcs, funcs_val, train_slope, eval_slope, v_true, strategic=False):
        torch.manual_seed(0)
        np.random.seed(0)

        super(MyStrategicModel, self).__init__()
        self.x_dim = x_dim
        self.train_slope, self.eval_slope = train_slope, eval_slope
        self.v_true = v_true
        self.v_train = v_true + 0.5*np.random.randn(x_dim)
        self.v_train -= (np.mean(self.v_train) - np.mean(v_true))
        self.v_train /= np.std(self.v_train)
        self.w = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(x_dim, dtype=torch.float64, requires_grad=True)))
        self.b = torch.nn.parameter.Parameter(torch.rand(1, dtype=torch.float64, requires_grad=True))
        self.strategic = strategic
        self.ccp = CCP(x_dim, funcs)
        self.ccp_val = CCP(x_dim, funcs_val)
        self.delta = DELTA(x_dim, funcs, self.v_train)

    def forward(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        v = self.v_true if evaluation else self.v_train
        ccp = self.ccp_val if evaluation else self.ccp
        
        if self.strategic:
            XT = ccp.optimize_X(X, self.w, self.b, slope, v)
            F_DER = self.get_f_ders(XT, slope)
            X_opt = self.delta.optimize_X(X, self.w, self.b, F_DER) # Xopt should equal to XT but we do it again for the gradients

            output = self.score(X_opt)
        else:
            output = self.score(X)        
        
        return output
    
    def optimize_X(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        v = self.v_true if evaluation else self.v_train
        ccp = self.ccp_val if evaluation else self.ccp
        return ccp.optimize_X(X, self.w, self.b, slope, v)
    
    def score(self, x):
        return x@self.w + self.b
    
    def get_f_ders(self, XT, slope):
        return torch.stack([0.5*slope*((slope*self.score(xt) + 1)/torch.sqrt((slope*self.score(xt) + 1)**2 + 1))*self.w for xt in XT])

    def evaluate(self, X, Y):
        Y_pred = torch.sign(self.forward(X, evaluation=True))
        num = len(Y)
        temp = Y - Y_pred
        acc = len(temp[temp == 0])*1./num        
        return acc
    
    def loss(self, Y, Y_pred):
        return torch.mean(torch.clamp(1 - Y_pred * Y, min=0))
    
    def save_model(self, train_errors, val_errors, train_losses, val_losses, info, path, comment=None):
        if comment is not None:
            path += "_____" + comment
            
        filename = path + "/model.pt"
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        torch.save(self.state_dict(), filename)
        
        pd.DataFrame(self.v_train).to_csv(path + '/v_train.csv')
        
        pd.DataFrame(np.array(train_errors)).to_csv(path + '/train_errors.csv')
        pd.DataFrame(np.array(val_errors)).to_csv(path + '/val_errors.csv')
        pd.DataFrame(np.array(train_losses)).to_csv(path + '/train_losses.csv')
        pd.DataFrame(np.array(val_losses)).to_csv(path + '/val_losses.csv')
        
        with open(path + "/info.txt", "w") as f:
            f.write(info)
    
    def load_model(self, filename):
        self.load_state_dict(torch.load(filename))
        self.eval()
    
    def fit(self, X, Y, Xval, Yval, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, calc_train_errors=False, comment=None):
        train_dset = TensorDataset(X, Y)
        train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        opt = opt(self.parameters(), **opt_kwargs)

        train_losses = []
        val_losses = []
        train_errors = []
        val_errors = []
        
        best_val_error = 1
        consecutive_no_improvement = 0
        now = datetime.now()
        path = "C:/Users/sagil/Desktop/nir project/models/hardt/" + now.strftime("%d-%m-%Y_%H-%M-%S")

        total_time = time.time()
        for epoch in range(epochs):
            t1 = time.time()
            batch = 1
            train_losses.append([])
            train_errors.append([])
            for Xbatch, Ybatch in train_loader:
                try:
                    opt.zero_grad()
                    Ybatch_pred = self.forward(Xbatch)
                    l = self.loss(Ybatch_pred, Ybatch)
                    l.backward()
                    opt.step()
                    train_losses[-1].append(l.item())
                    if calc_train_errors:
                        with torch.no_grad():
                            e = self.evaluate(Xbatch, Ybatch)
                            train_errors[-1].append(1-e)
                    if verbose:
                        print("batch %03d / %03d | loss: %3.5f" %
                              (batch, len(train_loader), np.mean(train_losses[-1])))
                    batch += 1
                    if callback is not None:
                        callback()
                except:
                    print("Failed")

            with torch.no_grad():
                try:
                    Yval_pred = self.forward(Xval)
                    val_loss = self.loss(Yval_pred, Yval).item()
                    val_losses.append(val_loss)
                    val_error = 1-self.evaluate(Xval, Yval)
                    val_errors.append(val_error)
                    if val_error < best_val_error:
                        consecutive_no_improvement = 0
                        best_val_error = val_error
                        if self.strategic:
                            info = "training time in seconds: {}\nepoch: {}\nbatch size: {}\ntrain slope: {}\neval slope: {}\nlearning rate: {}\nvalidation loss: {}\nvalidation error: {}\n".format(
                            time.time()-total_time, epoch, batch_size, self.train_slope, self.eval_slope, opt_kwargs["lr"], val_loss, val_error)
                            self.save_model(train_errors, val_errors, train_losses, val_losses, info, path, comment)
                            print("model saved!")
                
                    else:
                        consecutive_no_improvement += 1
                        if consecutive_no_improvement >= 4:
                            break
                except:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= 4:
                        break
                        
            t2 = time.time()
            if verbose:
                print("----- epoch %03d / %03d | time: %03d sec | loss: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_errors[-1]))
        print("training time: {} seconds".format(time.time()-total_time)) 
        return train_errors, val_errors, train_losses, val_losses

# Train

In [20]:
EPOCHS = 10
BATCH_SIZE = 96

x_dim = XDIM
v_true = np.array([-1,-1,-1,-1,-1,-1,-1,1,1,0.1,1,0.1,0.1,1,0.1])


# non-strategic classification
print("---------- training non-strategically----------")
non_strategic_model = MyStrategicModel(x_dim, funcs, funcs_val, TRAIN_SLOPE, EVAL_SLOPE, v_true, strategic=False)

fit_res_non_strategic = non_strategic_model.fit(X, Y, Xval, Yval,
                                opt=torch.optim.Adam, opt_kwargs={"lr": 5*(1e-2)},
                                batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=True, calc_train_errors=False)




gaming_list = np.arange(0, 3.1, 3/10)
gaming_list[0] = 0.1
print(gaming_list)
for t in gaming_list:
    print("training on t:{}".format(t))
    GAMING = t

    strategic_model = MyStrategicModel(x_dim, funcs, funcs_val, TRAIN_SLOPE, EVAL_SLOPE, v_true, strategic=True)
    print(strategic_model.v_train)
    fit_res_strategic = strategic_model.fit(X, Y, Xval, Yval,
                        opt=torch.optim.Adam, opt_kwargs={"lr": (1e-1)},
                        batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=True, calc_train_errors=False,
                        comment="hardt_gaming_" + str(t))
    print(strategic_model.v_train)
    
# # strategic classification
# print("---------- training strategically----------")
# strategic_model = MyStrategicModel(x_dim, funcs, funcs_val, TRAIN_SLOPE, EVAL_SLOPE, v_true, strategic=True)

# fit_res_strategic = strategic_model.fit(X, Y, Xval, Yval,
#                                 opt=torch.optim.Adam, opt_kwargs={"lr": (1e-1)},
#                                 batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=True, calc_train_errors=False,
#                                 comment="final_hardt_comparison_experiments")

---------- training non-strategically----------
1
1 0.1
1
batch 001 / 045 | loss: 1.08089
batch 002 / 045 | loss: 0.92122
batch 003 / 045 | loss: 0.82528
batch 004 / 045 | loss: 0.79352
batch 005 / 045 | loss: 0.75882
batch 006 / 045 | loss: 0.74093
batch 007 / 045 | loss: 0.70748
batch 008 / 045 | loss: 0.68846
batch 009 / 045 | loss: 0.71089
batch 010 / 045 | loss: 0.68390
batch 011 / 045 | loss: 0.67015
batch 012 / 045 | loss: 0.65939
batch 013 / 045 | loss: 0.65846
batch 014 / 045 | loss: 0.64749
batch 015 / 045 | loss: 0.64006
batch 016 / 045 | loss: 0.62427
batch 017 / 045 | loss: 0.60900
batch 018 / 045 | loss: 0.59651
batch 019 / 045 | loss: 0.59249
batch 020 / 045 | loss: 0.58386
batch 021 / 045 | loss: 0.57921
batch 022 / 045 | loss: 0.57635
batch 023 / 045 | loss: 0.56864
batch 024 / 045 | loss: 0.56108
batch 025 / 045 | loss: 0.55333
batch 026 / 045 | loss: 0.55022
batch 027 / 045 | loss: 0.54248
batch 028 / 045 | loss: 0.53817
batch 029 / 045 | loss: 0.52759
batch 030 / 04

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l


batch 038 / 045 | loss: 0.49572
batch 039 / 045 | loss: 0.49391
batch 040 / 045 | loss: 0.49362
batch 041 / 045 | loss: 0.49018
batch 042 / 045 | loss: 0.48654
batch 043 / 045 | loss: 0.48514
batch 044 / 045 | loss: 0.48271
batch 045 / 045 | loss: 0.47968
----- epoch 001 / 010 | time: 000 sec | loss: 0.40680 | err: 0.18869
batch 001 / 045 | loss: 0.23435
batch 002 / 045 | loss: 0.28169
batch 003 / 045 | loss: 0.35162
batch 004 / 045 | loss: 0.35439
batch 005 / 045 | loss: 0.35422
batch 006 / 045 | loss: 0.35565
batch 007 / 045 | loss: 0.35494
batch 008 / 045 | loss: 0.35191
batch 009 / 045 | loss: 0.35846
batch 010 / 045 | loss: 0.37304
batch 011 / 045 | loss: 0.38808
batch 012 / 045 | loss: 0.39539
batch 013 / 045 | loss: 0.39463
batch 014 / 045 | loss: 0.39365
batch 015 / 045 | loss: 0.39355
batch 016 / 045 | loss: 0.39204
batch 017 / 045 | loss: 0.39202
batch 018 / 045 | loss: 0.38400
batch 019 / 045 | loss: 0.38771
batch 020 / 045 | loss: 0.37820
batch 021 / 045 | loss: 0.37869
ba

batch 017 / 045 | loss: 0.34950
batch 018 / 045 | loss: 0.34480
batch 019 / 045 | loss: 0.34705
batch 020 / 045 | loss: 0.35390
batch 021 / 045 | loss: 0.35433
batch 022 / 045 | loss: 0.35198
batch 023 / 045 | loss: 0.35342
batch 024 / 045 | loss: 0.34982
batch 025 / 045 | loss: 0.35361
batch 026 / 045 | loss: 0.36422
batch 027 / 045 | loss: 0.36576
batch 028 / 045 | loss: 0.36872
batch 029 / 045 | loss: 0.36988
batch 030 / 045 | loss: 0.37208
batch 031 / 045 | loss: 0.37303
batch 032 / 045 | loss: 0.37382
batch 033 / 045 | loss: 0.37181
batch 034 / 045 | loss: 0.37697
batch 035 / 045 | loss: 0.37716
batch 036 / 045 | loss: 0.37588
batch 037 / 045 | loss: 0.38067
batch 038 / 045 | loss: 0.38045
batch 039 / 045 | loss: 0.37931
batch 040 / 045 | loss: 0.37776
batch 041 / 045 | loss: 0.37514
batch 042 / 045 | loss: 0.37629
batch 043 / 045 | loss: 0.37607
batch 044 / 045 | loss: 0.37577
batch 045 / 045 | loss: 0.37726
----- epoch 007 / 010 | time: 000 sec | loss: 0.38263 | err: 0.17809
bat

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l

...

batch 007 / 045 | loss: 1.06697
batch 008 / 045 | loss: 1.04312
batch 009 / 045 | loss: 1.06035
batch 010 / 045 | loss: 1.03188
batch 011 / 045 | loss: 1.02304
batch 012 / 045 | loss: 1.01183
batch 013 / 045 | loss: 1.00278
batch 014 / 045 | loss: 0.99580
batch 015 / 045 | loss: 0.98886
batch 016 / 045 | loss: 0.97713
batch 017 / 045 | loss: 0.97485
batch 018 / 045 | loss: 0.97088
batch 019 / 045 | loss: 0.96839
batch 020 / 045 | loss: 0.96600
batch 021 / 045 | loss: 0.95958
batch 022 / 045 | loss: 0.95032
batch 023 / 045 | loss: 0.94239
batch 024 / 045 | loss: 0.93481
batch 025 / 045 | loss: 0.93355
batch 026 / 045 | loss: 0.93019
batch 027 / 045 | loss: 0.92600
batch 028 / 045 | loss: 0.91992
batch 029 / 045 | loss: 0.91363
batch 030 / 045 | loss: 0.91273
batch 031 / 045 | loss: 0.92156
batch 032 / 045 | loss: 0.92245
batch 033 / 045 | loss: 0.92139
batch 034 / 045 | loss: 0.91789
batch 035 / 045 | loss: 0.91239
batch 036 / 045 | loss: 0.90601
batch 037 / 045 | loss: 0.90556
batch 03

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l

batch 001 / 045 | loss: 1.71434
batch 002 / 045 | loss: 1.34808
batch 003 / 045 | loss: 1.15791
batch 004 / 045 | loss: 1.14238
batch 005 / 045 | loss: 1.10716
batch 006 / 045 | loss: 1.09612


C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\problems\problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


batch 007 / 045 | loss: 1.08172
batch 008 / 045 | loss: 1.05738
batch 009 / 045 | loss: 1.07123
batch 010 / 045 | loss: 1.04412
batch 011 / 045 | loss: 1.03580
batch 012 / 045 | loss: 1.02476
batch 013 / 045 | loss: 1.01599
batch 014 / 045 | loss: 1.00958
batch 015 / 045 | loss: 1.00225
batch 016 / 045 | loss: 0.98980
batch 017 / 045 | loss: 0.98814
batch 018 / 045 | loss: 0.98595
batch 019 / 045 | loss: 0.98359
batch 020 / 045 | loss: 0.98113
batch 021 / 045 | loss: 0.97428
batch 022 / 045 | loss: 0.96463
batch 023 / 045 | loss: 0.95665
batch 024 / 045 | loss: 0.94893
batch 025 / 045 | loss: 0.94681
batch 026 / 045 | loss: 0.94323
batch 027 / 045 | loss: 0.93846
batch 028 / 045 | loss: 0.93267
batch 029 / 045 | loss: 0.92559
batch 030 / 045 | loss: 0.92479
batch 031 / 045 | loss: 0.93173
batch 032 / 045 | loss: 0.92998
batch 033 / 045 | loss: 0.92777
batch 034 / 045 | loss: 0.92652
batch 035 / 045 | loss: 0.91962
batch 036 / 045 | loss: 0.91550
batch 037 / 045 | loss: 0.91620
batch 03

# Test results

In [ ]:
# Xval_opt = non_strategic_model.optimize_X(Xval, evaluation=True)
# print("non strategic model + non strategic data: ", non_strategic_model.evaluate(Xval, Yval))
# print("non strategic model + strategic data: ", non_strategic_model.evaluate(Xval_opt, Yval))
# print("strategic model + strategic data: ", strategic_model.evaluate(Xval, Yval))
